In [ ]:
from mediawiki import MediaWiki
from flair.data import Sentence
from flair.models import SequenceTagger
from transformers import pipeline



class WikiFinder:
    def __init__(self) -> None:
        self.tagger = SequenceTagger.load('flair/ner-english')
        self.wikipedia = MediaWiki()
    
    def ner_predict(self, question):
        sentence = Sentence(question)
        self.tagger.predict(sentence)
        entities = [entity for entity in sentence.get_spans('ner')]
        return entities

    def find_wikis(self, word):
        if not isinstance(word, str):
            word = word.to_original_text()
        return self.wikipedia.search(word)

    def read_content(self, page):
        return self.wikipedia.page(page).content

class FindAnswers:
    def __init__(self):
        self.nlp = pipeline("question-answering")
        self.wf = WikiFinder()

    def ExtractText(self, question, context):
        return self.nlp(question=question, context=context)

    def AnswerQuestion(self, question):
        print(f"Answering qn :{question}")
        entities = self.wf.ner_predict(question)
        wikis = []
        answers = []
        for entity in entities:
            wikis += self.wf.find_wikis(entity)[0:1]
        for wiki in wikis:
            print(f"searching wiki: {wiki}")
            context = self.wf.read_content(wiki)
            if context:
                answer = self.ExtractText(question, context)
                print(answer)
                answers.append(answer)
        return answers


In [ ]:
fa = FindAnswers()

In [ ]:
fa.AnswerQuestion("What was the occupation of Lovely Rita according to the song by the Beatles")